In [1]:
import pandas as pd
import json
import glob
import os
pd.options.mode.chained_assignment = None
path = 'C:\\Users\\38164\\Downloads\\jsonl\\'
key_columns = ['living_space','floor', 'rooms', 'zip_code', 'sale_type', 'price']

In [47]:
def read_jsonl(path):
    df_ = pd.DataFrame(index=None)
    path=path
    extension = 'jsonl'
    os.chdir(path)
    files = glob.glob('*.{}'.format(extension))
    print("files to load: " + str(len(files)))
    

    for file in files:  
        appartments=[]
        with open(file, 'r') as json_file:
            json_list_home = list(json_file)
            print("loading file: " + file)
        for json_str in json_list_home:
                result = json.loads(json_str)
                appartments.append(result)
        df = pd.DataFrame(appartments)
        df['platform'] = file.split('.')[0]
        frames = [df_, df]
        df_ = pd.concat(frames)
    print('rows loaded: ' + str(len(df_)))
    
    return df_

def _get_df_with_duplicates(df,key_columns):
    key_columns=key_columns
    df = df
    #we drop those rows that have to many nan values in key columns
    df.dropna(subset=key_columns, thresh=4,inplace=True)
    duplicate_filter = df.duplicated(subset=key_columns, keep=False)
    df_duplicates = df[duplicate_filter]
       
    return df_duplicates

def _get_key_column(df, key_columns):
    key_columns=key_columns
    df=df
    
    for c in key_columns:
        df[c] = df[c].astype(str)
    df['key']=df[key_columns].T.agg(','.join)
        
    return df

def df_to_list(df, key_columns):
    df=df
    key_columns=key_columns
    df=_get_key_column(df, key_columns)
    df = df[['id','platform','key']]
    grouped = df.groupby(df['key'])
    list_of_duplicates = []
    for key in df['key'].unique():
        temporary_df = grouped.get_group(key)[['platform','id']]
        l = temporary_df.to_dict('records')
        list_of_duplicates.append(l)
        
    return list_of_duplicates

In [53]:
df=read_jsonl(path)

files to load: 2
loading file: homegate.jsonl
loading file: immoscout24.jsonl
rows loaded: 147458


In [54]:
df

,id,title,description,price,category,sale_type,images,living_space,floor,rooms,zip_code,city,url,platform
0,3001713521,confortable 2.5 pièces neuf,confortable logement 2.5 pièces traversant ave...,"360,000.00",apartment,buy,[https://media2.homegate.ch/listings/g133/3001...,55.0,NaN,2.5,1680,Romont FR,https://www.homegate.ch/kaufen/3001713521,homegate
1,3001094427,"Dpt Jura (39), à vendre proche de ORCHAMPS mai...",Secteur Orchamps ;<br /><br />Venez découvrir ...,"315,000.00",house,buy,[https://media2.homegate.ch/listings/capi01/30...,280.0,NaN,11.0,39700,PLUMONT,https://www.homegate.ch/kaufen/3001094427,homegate
2,3001634047,GROSSZÜGIG AN ZENTRALER LAGE,Diese lichtdurchflutete 3.5-Zimmer-Wohnung bef...,"1,305.00",apartment,rent,[https://media2.homegate.ch/listings/z409casa/...,NaN,1.0,3.5,9030,Abtwil SG,https://www.homegate.ch/mieten/3001634047,homegate
3,3001774934,Ufficio in palazzo rappresentativo in centro c...,Affittasi in palazzo rappresentativo ufficio c...,"2,370.00",commercial,rent,[https://media2.homegate.ch/listings/gpmv2/300...,63.0,4.0,4.0,6900,Paradiso,https://www.homegate.ch/mieten/3001774934,homegate
4,3000829877,Ferme authentique et historique avec installat...,Cette splendide maison paysanne classée aux bi...,"2,550,000.00",house,buy,[https://media2.homegate.ch/listings/a548/3000...,316.0,NaN,8.5,1724,Le Mouret,https://www.homegate.ch/kaufen/3000829877,homegate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68721,6498569,Gemütliche Wohnung nahe Dorfplatz,\r\nAn der Deitingenstrasse 8 haben wir eine g...,"1,040.00",apartment,rent,[https://cis01.immoscout24.ch/is24media/cb/45/...,NaN,2.0,3.5,4542,Luterbach,https://www.immoscout24.ch/6498569,immoscout24
68722,7024843,Appartement à louer,"Etat de neuf, non-fumeur et directement du pro...","2,590.00",apartment,rent,[https://cis01.immoscout24.ch/is24media/38/9b/...,87.0,NaN,3.5,1029,Villars-Ste-Croix,https://www.immoscout24.ch/7024843,immoscout24
68723,6362716,Local /Cabinet/ bureau à louer 660.-,Local commercial/cabinet/bureau \r\nPossibil...,660.00,commercial,rent,[https://cis01.immoscout24.ch/is24media/42/c7/...,NaN,NaN,1.0,1029,Villars-Ste-Croix,https://www.immoscout24.ch/6362716,immoscout24
68724,6946171,"Magnifique appartement avec jardin, proche des...",\r\nSitué au cœur du village calme et verdoyan...,"2,430.00",apartment,rent,[https://cis01.immoscout24.ch/is24media/db/f0/...,72.0,0.0,3.5,1029,Villars-Ste-Croix,https://www.immoscout24.ch/6946171,immoscout24


In [68]:
df_d = _get_df_with_duplicates(df)
df=_get_key_column(df_d, key_columns)
len(df_d)

72648

In [56]:
df_d.sort_values(key_columns)

,id,title,description,price,category,sale_type,images,living_space,floor,rooms,zip_code,city,url,platform
45198,3001703608,Günstig Wohnen am Bodensee - 1. Mietzins gesch...,* Bei Abschluss eines Jahresvertrages schenken...,950.00,apartment,rent,[https://media2.homegate.ch/listings/liv2/3001...,1.0,0.0,3.0,9400,Rorschach,https://www.homegate.ch/mieten/3001703608,homegate
47131,6347233,Günstig Wohnen am Bodensee - 1. Mietzins gesch...,\* Bei Abschluss eines Jahresvertrages schenke...,950.00,apartment,rent,[https://cis01.immoscout24.ch/is24media/f3/96/...,1.0,0.0,3.0,9400,Rorschach,https://www.immoscout24.ch/6347233,immoscout24
16526,3000999289,Ihre neue Gewerbefläche - vielfältig nutzbar,Per sofort oder nach Vereinbarung steht diese ...,None,commercial,rent,[https://media2.homegate.ch/listings/wic/30009...,1.0,0.0,NaN,8207,Schaffhausen,https://www.homegate.ch/mieten/3000999289,homegate
25471,6958848,Ihre neue Gewerbefläche - vielfältig nutzbar,Per sofort oder nach Vereinbarung steht diese ...,None,commercial,rent,[https://cis01.immoscout24.ch/is24media/a2/2f/...,1.0,0.0,NaN,8207,Schaffhausen,https://www.immoscout24.ch/6958848,immoscout24
1412,3001757417,PLUS DISPONIBLE,Magnifique Restaurant idéalement situé dans un...,None,commercial,buy,[],1.0,1.0,1.0,1095,Lutry,https://www.homegate.ch/kaufen/3001757417,homegate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42238,6909556,Self Check-in Motel an guter Lage,\r\nHier haben Sie die Möglichkeit ein Motel m...,"5,000,000.00",commercial,buy,[https://cis01.immoscout24.ch/is24media/f9/df/...,NaN,NaN,32.0,3295,Rüti b. Büren,https://www.immoscout24.ch/6909556,immoscout24
12318,3001737515,"SITUATION IDEAL, MAGNIFIQUE HÔTEL",Fonds de commerce d'un magnifique Hôtel trois ...,"1,100,000.00",commercial,buy,[https://media2.homegate.ch/listings/x379/3001...,NaN,NaN,35.0,1228,Plan-les-Ouates,https://www.homegate.ch/kaufen/3001737515,homegate
8040,6978765,"SITUATION IDEAL, MAGNIFIQUE HÔTEL",Fonds de commerce d'un magnifique Hôtel trois ...,"1,100,000.00",commercial,buy,[https://cis01.immoscout24.ch/is24media/45/0e/...,NaN,NaN,35.0,1228,Plan-les-Ouates,https://www.immoscout24.ch/6978765,immoscout24
54603,3001754198,HÔTEL 5 ÉTOILES,Projet d'un magnifique Hôtel d'environs 300 ch...,"80,000,000.00",commercial,buy,[https://media2.homegate.ch/listings/x379/3001...,NaN,NaN,310.0,8152,Opfikon,https://www.homegate.ch/kaufen/3001754198,homegate


In [63]:
#df[df['key']=='316.0,nan,12.0,3068,buy,1,795,000.00']
df[df['key']=='55.0,nan,2.5,1680,buy,360,000.00']

,id,title,description,price,category,sale_type,images,living_space,floor,rooms,zip_code,city,url,platform,key
0,3001713521,confortable 2.5 pièces neuf,confortable logement 2.5 pièces traversant ave...,"360,000.00",apartment,buy,[https://media2.homegate.ch/listings/g133/3001...,55.0,nan,2.5,1680,Romont FR,https://www.homegate.ch/kaufen/3001713521,homegate,"55.0,nan,2.5,1680,buy,360,000.00"
52698,3001713551,confortable 2.5 pièces neuf,confortable logement 2.5 pièces traversant ave...,"360,000.00",apartment,buy,[https://media2.homegate.ch/listings/g133/3001...,55.0,nan,2.5,1680,Romont FR,https://www.homegate.ch/kaufen/3001713551,homegate,"55.0,nan,2.5,1680,buy,360,000.00"


In [58]:
final_list=df_to_list(df)
final_list[0]

[{'platform': 'homegate', 'id': '3001713521'},
 {'platform': 'homegate', 'id': '3001713551'}]

In [46]:
with open('list_output.json', "w") as fp:
        json.dump(final_list, fp, indent=4)

In [64]:
final_list

[[{'platform': 'homegate', 'id': '3001713521'},
  {'platform': 'homegate', 'id': '3001713551'}],
 [{'platform': 'homegate', 'id': '3001634047'},
  {'platform': 'immoscout24', 'id': '6881811'}],
 [{'platform': 'homegate', 'id': '3000829877'},
  {'platform': 'immoscout24', 'id': '4246888'},
  {'platform': 'immoscout24', 'id': '5793620'}],
 [{'platform': 'homegate', 'id': '3001488644'},
  {'platform': 'immoscout24', 'id': '6778455'},
  {'platform': 'immoscout24', 'id': '6640668'}],
 [{'platform': 'homegate', 'id': '3001187149'},
  {'platform': 'immoscout24', 'id': '6564459'}],
 [{'platform': 'homegate', 'id': '3001773602'},
  {'platform': 'immoscout24', 'id': '7017304'}],
 [{'platform': 'homegate', 'id': '3001527187'},
  {'platform': 'immoscout24', 'id': '6808690'}],
 [{'platform': 'homegate', 'id': '3001767071'},
  {'platform': 'immoscout24', 'id': '7008672'}],
 [{'platform': 'homegate', 'id': '3001734636'},
  {'platform': 'immoscout24', 'id': '6977047'}],
 [{'platform': 'homegate', 'id'

In [66]:
df[df['id']=='3001678318']['key']

38    78.0,4.0,3.5,1950,rent,1,500.00
Name: key, dtype: object

In [67]:
df[df['key']=='78.0,4.0,3.5,1950,rent,1,500.00']

,id,title,description,price,category,sale_type,images,living_space,floor,rooms,zip_code,city,url,platform,key
38,3001678318,Appartement moderne de 3.5pces,"A LOUER à Sion, proche de la gare et des accè...","1,500.00",apartment,rent,[https://media2.homegate.ch/listings/sv100/300...,78.0,4.0,3.5,1950,Sion,https://www.homegate.ch/mieten/3001678318,homegate,"78.0,4.0,3.5,1950,rent,1,500.00"
52495,3001653392,Situation centrale à deux pas de la gare,"Idéalement situé au centre de Sion, cet immeub...","1,500.00",apartment,rent,[https://media2.homegate.ch/listings/liv2/3001...,78.0,4.0,3.5,1950,Sion,https://www.homegate.ch/mieten/3001653392,homegate,"78.0,4.0,3.5,1950,rent,1,500.00"
66459,6923350,Appartement moderne de 3.5pces,"A LOUER à Sion, proche de la gare et des accès...","1,500.00",apartment,rent,[https://cis01.immoscout24.ch/is24media/99/24/...,78.0,4.0,3.5,1950,Sion,https://www.immoscout24.ch/6923350,immoscout24,"78.0,4.0,3.5,1950,rent,1,500.00"
66460,6312698,Situation centrale à deux pas de la gare,"Idéalement situé au centre de Sion, cet immeub...","1,500.00",apartment,rent,[https://cis01.immoscout24.ch/is24media/10/41/...,78.0,4.0,3.5,1950,Sion,https://www.immoscout24.ch/6312698,immoscout24,"78.0,4.0,3.5,1950,rent,1,500.00"


In [52]:
len(final_list)

34108

In [59]:
df = df[['id','platform','key']]
df

,id,platform,key
0,3001713521,homegate,"55.0,nan,2.5,1680,buy,360,000.00"
2,3001634047,homegate,"nan,1.0,3.5,9030,rent,1,305.00"
4,3000829877,homegate,"316.0,nan,8.5,1724,buy,2,550,000.00"
6,3001488644,homegate,"107.0,0.0,4.5,1920,buy,640,000.00"
8,3001187149,homegate,"125.0,6.0,6.5,6600,buy,790,000.00"
...,...,...,...
68711,6838902,immoscout24,"70.0,2.0,2.5,5626,rent,1,400.00"
68712,6751095,immoscout24,"90.0,2.0,4.5,5630,rent,1,730.00"
68715,5874654,immoscout24,"86.0,1.0,4.5,5630,rent,1,740.00"
68716,7015399,immoscout24,"66.0,nan,2.5,5630,rent,1,415.00"


In [ ]:
df = df[['id','platform','key']]
grouped = df.groupby(df['key'])
list_of_duplicates = []
for key in df['key'].unique():
    temporary_df = grouped.get_group(key)[['platform','id']]
    l = temporary_df.to_dict('records')
    list_of_duplicates.append(l)